## useTransition() hook

In the below example there are three tabs. Products tab generates fake 15k records to dispaly and is delayed by 1.5s in total. When you click on the Products tab and then quickly on About, after 1.5s you will not see any change of the style on Products button, you will only see changed style on About button. 

With useTransition() hook we can "tell" React which state change can be interrupted, so we don't have to wait 1.5s to see the state change.

In [ ]:
## styles.css

body {
    font-family: Arial, Helvetica, sans-serif;
    background: #737373;
    color: white;
}

nav {
    text-align: center;
}

h1 {
    margin-top: 40px;
}

button {
    padding: .5rem 1rem;
    border: 1px solid black;
    border-radius: 5px;
    margin-inline: 5px;
    cursor: pointer;
}

button:focus {
    outline: none;
}

button:focus-visible {
    outline: 5px solid Highlight;
}

In [ ]:
## data.js

# fake data (install faker first!

import { faker } from "@faker-js/faker"

enum Color { Red = "Red", Green = "Green", Blue = "Blue" }

const products = new Array(1500).fill(null).map(() => ({
    id: faker.database.mongodbObjectId(),
    name: faker.commerce.productName(),
    description: faker.commerce.productDescription(),
    material: faker.commerce.productMaterial(),
    price: faker.commerce.price(),
    department: faker.commerce.department(),
    quantity: faker.number.int({min: 0, max: 100}),
    color: faker.helpers.enumValue(Color)
}))

export default products

In [ ]:
## utils.js

## function to delay display of products
export function sleep(ms) {
    const wakeUpTime = Date.now() + ms;
    while(Date.now() < wakeUpTime) {}
  }

In [ ]:
## main component

'use client'

import products from '@/lib/data';
import React, { memo, useState, useTransition } from 'react'; ## import useTransition hook
import {sleep} from "@/lib/utils"
import "./styles.css"

const Transition = () => {
  const [tab, setTab] = useState("home")
    
  const [isPending, startTransition] = useTransition(); ## calling use transition hook
  ## The isPending flag tells you whether there is a pending Transition.

  function switchTab(tab: string) {
    startTransition(() => {
      setTab(tab)
    })
  }
    
  # *** BEFORE IT LOOKED LIKE BELOW ***
    
  # function switchTab(tab: string) {
  #       setTab(tab)
  #   }
    

  function setStyles(thisTab: string) {
    return {
      backgroundColor: tab === thisTab ? "#262626" : "white",
      color: tab === thisTab ? "white" : "black"
    }
  }

  return (
    <main>
      <nav>
        <button
          onClick={() => switchTab("home")}
          style={setStyles("home")}
        >Home</button>
        <button
          onClick={() => switchTab("products")}
          style={setStyles("products")}
        >Products</button>
        <button
          onClick={() => switchTab("about")}
          style={setStyles("about")}
        >About</button>
      </nav>
      <div>
        {isPending && <p>Loading...</p>} ## we can use isPending flag here, to display loading
      ## information to the user
      
        {!isPending && tab === "home" && <h1>Home page</h1>}
        {!isPending && tab === "products" && <Products />}
        {!isPending && tab === "about" && <h1>About page</h1>}
      </div>
    </main>
  )
}


const Products = memo(function () {
  const productsList = products.map((product, i) => (
    <SlowProduct key={product.id} product={product} />
  ))

  return (
    <>
      <h1>Products page</h1>
      <ul>{productsList}</ul>
    </>
  )
});

function SlowProduct({ product }: {product: {name: string}}) {
  sleep(1)
  return <li>Product {product.name}</li>
}

export default Transition